# Multiprocessing in Python
### Author: https://github.com/Tsangares
Common pitfalls of multiprocessing with python.

### The Process and Queue
First we do the classic process and queue; the queue is 

In [1]:
from multiprocessing import Queue,Process

Lets make the computaiton to do will be a random walk.
We are going to use `numpy` to do this. 

In [2]:
import numpy as np
import random,time

In [3]:
def random_walk(dimention=3,iterations=10_000):
    location = np.zeros(dimention)
    for i in range(iterations):
        location += np.array(random.choices([-1,1],k=dimention))
    print(f"We landed at {location}")
    return location
start = time.time()
random_walk(4,100)
print(f'It took {time.time() - start:.06f} secconds to generate this random walk.')

We landed at [ -2. -30.  -8.  16.]
It took 0.005899 secconds to generate this random walk.


 1. Make a list to contain all the processes
 2. Make the processes
 3. Run the processes
 4. Wait for them

In [4]:
processes = []
for _ in range(4):
    p = Process(target=random_walk)
    processes.append(p)
    p.start()
for p in processes:
    p.join()
print("Done.")

We landed at [ 26. -48. -10.]
We landed at [ 10. -18. -62.]
We landed at [ 80.  54. 120.]
We landed at [-50. -28.  56.]
Done.


### Wait I want the results?
Thats why we need a queue

In [5]:
queue = Queue()

#Wrap the function to add the results in a queue
random_walk_with_queue = lambda d=3,k=10_000: queue.put(random_walk(d,k))

#Change the target!
processes = []
for _ in range(4):
    p = Process(target=random_walk_with_queue)
    processes.append(p)
    p.start()
for p in processes:
    p.join()
print("Done.")

We landed at [ 10.  84. -34.]
We landed at [118. -14.  -2.]
We landed at [-58.  72. 124.]
We landed at [ 86.  18. -34.]
Done.


Now lets unpack the queue
print(queue)

In [6]:
print(queue)
walks = []
while not queue.empty():
    walks.append(queue.get())
    
print(walks)

[array([ 10.,  84., -34.]), array([118., -14.,  -2.]), array([-58.,  72., 124.]), array([ 86.,  18., -34.])]


## Yo that all kinda sucks... is there a better way
### Yes POOL
Now we do the same using a multiprocessing pool 

In [7]:
from multiprocessing import Pool

With 4 processes get 100 random walks in 4 dimentions with 10,000 iterations. But now my function needs an iterator to give it an index (tell it when to stop).

In [8]:
def random_walk_indexed(index,dimention,iterations):
    return random_walk(dimention,iterations)

In [9]:
#The second argument in starmap is the parameters
from itertools import repeat

dimention = 4
iterations = 10_000
vectors = 100

with Pool(4) as pool:
    vecs = pool.starmap(random_walk_indexed, zip(range(vectors),repeat(dimention),repeat(vectors)))

We landed at [ -6. -16.   4.   2.]
We landed at [ 16.   8. -16. -10.]
We landed at [ 2. -2. -8.  0.]
We landed at [-6.  2. -4.  2.]
We landed at [ -2.   0.  -2. -12.]
We landed at [ 10.   4. -20.  -2.]
We landed at [-14. -12.  10.   2.]
We landed at [ 4. -2.  0.  8.]
We landed at [-16.  -6.   0. -16.]
We landed at [10.  8. -8. -4.]
We landed at [10. 20. 14.  2.]
We landed at [ 6.  4. -4. 12.]
We landed at [10.  8. 14. -2.]
We landed at [-18.  16.   2.  -8.]
We landed at [-18.  -8.  10.   8.]
We landed at [-8. -2. -6. 16.]
We landed at [12.  6. -6.  0.]
We landed at [ -4.  -2. -10.  -2.]
We landed at [18. -2. -6.  4.]
We landed at [ 4. -2.  2. -8.]
We landed at [ 4. -2. -6. -6.]
We landed at [  4. -22.   6.  12.]
We landed at [-10. -12.  -6.  16.]
We landed at [20. -8. -6. 18.]
We landed at [-18.   4.   0.  -2.]
We landed at [ -6. -12.   4.   2.]
We landed at [12.  4. -8.  2.]
We landed at [22. -4. -6.  2.]
We landed at [-4. 10. 18.  2.]
We landed at [-8.  4.  4.  8.]
We landed at [  8.

Now we can just look at our results:

In [10]:
print(vecs)

[array([ -6., -16.,   4.,   2.]), array([ -2.,   0.,  -2., -12.]), array([-16.,  -6.,   0., -16.]), array([10.,  8., 14., -2.]), array([12.,  6., -6.,  0.]), array([ 4., -2.,  2., -8.]), array([-18.,   4.,   0.,  -2.]), array([-6.,  2., -4.,  2.]), array([ 4., -2.,  0.,  8.]), array([10., 20., 14.,  2.]), array([-18.,  -8.,  10.,   8.]), array([18., -2., -6.,  4.]), array([-10., -12.,  -6.,  16.]), array([ -6., -12.,   4.,   2.]), array([ 16.,   8., -16., -10.]), array([-14., -12.,  10.,   2.]), array([ 6.,  4., -4., 12.]), array([-8., -2., -6., 16.]), array([  4., -22.,   6.,  12.]), array([12.,  4., -8.,  2.]), array([-8.,  4.,  4.,  8.]), array([ 2., -2., -8.,  0.]), array([ 10.,   4., -20.,  -2.]), array([10.,  8., -8., -4.]), array([-18.,  16.,   2.,  -8.]), array([ -4.,  -2., -10.,  -2.]), array([ 4., -2., -6., -6.]), array([20., -8., -6., 18.]), array([22., -4., -6.,  2.]), array([  0.,   8., -12.,   6.]), array([  6.,   4., -14.,   8.]), array([10.,  4.,  4., 16.]), array([14.,